In [1]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [2]:
#laod dataset yang akan digunakan untuk test
data = pd.read_csv('Data\sentiment.csv')

In [3]:
data

,reviews_content,new_category
0,airplane ! is considered among many to be the ...,1
1,you've got to love disney . \nno matter what t...,1
2,""" the tailor of panama "" is a different kind ...",1
3,"the characters in jonathan lynn's "" the whole ...",0
4,"vikings v . bears ? \nno , this isn't the line...",0
...,...,...
1495,"trekkies , roger nygard's energetic and hilari...",1
1496,""" dangerous beauty "" is a really nothing more...",1
1497,starring shawnee smith ; donovan leitch ; rick...,0
1498,"man , this was one wierd movie . \nsimilar to ...",0


In [4]:
data['reviews_content']

0       airplane ! is considered among many to be the ...
1       you've got to love disney . \nno matter what t...
2        " the tailor of panama " is a different kind ...
3       the characters in jonathan lynn's " the whole ...
4       vikings v . bears ? \nno , this isn't the line...
                              ...                        
1495    trekkies , roger nygard's energetic and hilari...
1496     " dangerous beauty " is a really nothing more...
1497    starring shawnee smith ; donovan leitch ; rick...
1498    man , this was one wierd movie . \nsimilar to ...
1499    review : ghost dog : the way of the samurai ( ...
Name: reviews_content, Length: 1500, dtype: object

In [5]:
reviews = data['reviews_content'][0]

reviews

'airplane ! is considered among many to be the epitome of satire film-making . \nafter all , it\'s brought to us by one of the best known satire writing/directing teams . \neven if most people don\'t recognize the names behind the films , they are bound to recognize the titles : airplane ! , top secret , the naked gun , and hot shots to name a few . \nbut although the zucker/abrahams/zucker team was first introduced with the kentucky fried movie in 1977 , airplane ! \nremains the true cornerstone of their work , and their directorial debuts . \nin the seventies , disaster films seemed to be at an all time high . \nfilms like earthquake , the towering inferno , and the poseidon adventure were big hits . \nthere was also a series about the disasters that can arise when traveling by plane - a series that spanned the entire decade . \nand so , in 1980 , we were introduced to a new airplane disaster film . \nthis time the disaster had nothing to do with a maniacal hijacker or crashing into 

In [30]:
def prepare_json(text):
    feature_spec = {
        "reviews_content": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(reviews)


    
endpoint = "http://localhost:8080/v1/models/sentiments-detection-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "positive" if prediction_value > 0.6 else "negative"
else:
    result = "Error: No predictions found in response."

print(result)

Error: No predictions found in response.
